In [1]:
import json
import uuid
from datetime import datetime, timedelta
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# --- Load environment variables ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

# --- Connect to MongoDB ---
client = MongoClient(uri)
db = client[database_name]
collection = db[collection_name]

# --- Load the previous document (latest by current_form_qr_code) ---
previous = collection.find_one(sort=[("current_form_qr_code", -1)])
if not previous:
    raise ValueError("No previous sales_invoices document found.")

# --- Compute next form QR code ---
prev_qr = previous["current_form_qr_code"]
next_date = datetime.strptime(prev_qr, "%Y%m%d") + timedelta(days=1)
next_qr = next_date.strftime("%Y%m%d")

# --- Build new schema-compliant document ---
new_doc = {
    "id": str(uuid.uuid4()),
    "previous_form_qr_code": prev_qr,
    "current_form_qr_code": next_qr,
    "tin": previous.get("tin", ""),
    "location": previous.get("location", ""),
    "created": datetime.utcnow().isoformat() + "Z",
    "date": next_date.strftime("%Y-%m-%d")
}

# --- Insert into MongoDB ---
collection.insert_one(new_doc)

print("✅ New empty sales_invoices form created following schema.")
print(json.dumps(new_doc, indent=2, default=str))


✅ New empty sales_invoices form created following schema.
{
  "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "20251013",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T05:03:46.557977Z",
  "date": "2025-10-13",
  "_id": "68ec8832dcfa49cf3d7e33ef"
}


C:\Users\jello\AppData\Local\Temp\ipykernel_7920\1083752317.py:36: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created": datetime.utcnow().isoformat() + "Z",
